# 读写文件

加载和保存张量

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')

x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表，然后把它们读回内存

In [2]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

写入或读取从字符串映射到张量的字典

In [3]:
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [4]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 隐藏层：全连接层，输入维度20，输出维度256
        self.hidden = nn.Linear(20, 256)
        # 输出层：全连接层，输入维度256，输出维度10
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        '''
        self.hidden(x)：通过隐藏层，形状从(20)→(256)
        F.relu(...)：应用ReLU激活函数，负数归零
        self.output(...)：通过输出层，形状从(256)→(10)
        '''
        return self.output(F.relu(self.hidden(x)))

net = MLP()
# 生成2个样本，每个样本20维特征 
X = torch.randn(size=(2, 20))
'''
输入 X 形状：(2, 20)
经过 hidden 层 + ReLU：形状 (2, 256)
经过 output 层：最终输出 Y 形状 (2, 10)
'''
Y = net(X)

将模型的参数存储在一个叫做“mlp.params”的文件中

In [5]:
torch.save(net.state_dict(), 'mlp.params')

实例化了原始多层感知机模型的一个备份。
直接读取文件中存储的参数

In [6]:
# 关键点: 加载参数前必须先实例化模型，因为load_state_dict()需要知道参数的结构和名称
clone = MLP()
# clone.load_state_dict(...): 将加载的参数复制到clone模型中
# 要求: 文件中的参数名和形状必须与clone的state_dict()完全匹配
clone.load_state_dict(torch.load('mlp.params'))
# 将模型切换到评估模式（eval mode）
# 禁用 Dropout（dropout层停止随机失活）
# 调整 BatchNorm: 使用全局统计均值/方差，而非当前批次统计
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [9]:
# 将相同的输入X传入clone模型进行前向计算
# clone 是加载了保存参数并处于评估模式的模型
Y_clone = clone(X)
# 逐元素比较两个输出张量是否相等
Y_clone == Y
# 只想知道是否完全相等
# torch.equal(Y_clone, Y)  # 返回单个布尔值 True/False

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])